# 版本2.0
- 模型：bert-base-chinese
- 数据集：蚂蚁金融语义相似度数据集 AFQMC 作为语料，提供了官方的数据划分，训练集 / 验证集 / 测试集分别包含 34334 / 4316 / 3861 个句子对，标签 0 表示非同义句，1 表示同义句：
- 数据集格式：json
- 任务：二分类
- 指标：accuracy,f1
### 特点：
- 手撕
- 定义数据集进行数据加载
- 继承了BertForSequenceClassification模型来写模型
- 定义了损失函数和优化器
- 定义了训练循环和评估循环

### 由于GPU的原因，只选取了训练集的前8000个样本进行训练，可以修改max_samples参数来改变训练样本的数量

In [1]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer,AutoModel
import json


d:\software\Anaconda\envs\LLM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. 定义数据集并进行一次性读取
- strip()的必要性在于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
大多数情况都会有：
line == '{"a": 1, "b": 2}\n'
- enumerate(f) 本质是：对文件对象按行迭代，同时给每一行加一个自增编号
- for idx, line in enumerate(f):




| 变量     | 含义             |
| ------ | -------------- |
| `idx`  | 当前是第几行（从 0 开始） |
| `line` | 当前这一行的字符串内容    |


In [2]:
class AFQMC(Dataset):
    def __init__(self,data_path,max_samples=5000):
        self.data = self.load_data(data_path,max_samples)
    def load_data(self,data_path,max_samples=5000):
        data = {}
        with open(data_path,'r',encoding='utf-8') as f:
            #以文本模式读取json 
            #单条加载
            for idx, line in enumerate(f):
                if idx >= max_samples:          # 只取前 max_samples 条
                    break
                #strip()方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
                #loads()传入的是一个字符串，返回一个python对象
                sample = json.loads(line.strip())
                data[idx] = sample
        return data
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        sample = self.data[idx]
        return sample
    
    

In [18]:
traindata=AFQMC('./afqmc_public/train.json',max_samples=8000)
#取训练数据的前3000条

valdata=AFQMC('./afqmc_public/dev.json',max_samples=3000)
#取验证数据的前1000条


##### 对于特别大的数据集，难以一次性加载到内存中
- 解决方法：使用可迭代数据集（IterableDataset）

In [77]:
from torch.utils.data import IterableDataset

class IterableAFQMC(IterableDataset):
    def __init__(self, data_file):
        self.data_file = data_file

    def __iter__(self):
        with open(self.data_file, 'rt') as f:
            for line in f:
                sample = json.loads(line.strip())
                yield sample

#train_data = IterableAFQMC('data/afqmc_public/train.json')

# 2. 加载到dataloader中并进行分词处理 
- 按 batch 切分后的“批次迭代器”dataloader 并预处理
- 需要把内容和标签都转为tensor
- 需要加载预训练的bert-base-chines的checkpoint作为分词器
- 每个batch包括input_ids, attention_mask, token_type_ids, labels四个特征，但每个特征的维度都是(batch_size, max_seq_len)
- max_seq_len是动态填充的

In [ ]:
from torch.utils.data import DataLoader

checkpoint = "bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def collate_fn(batch):
    batch_sen1=[]
    batch_sen2=[]
    batch_labels=[]
    for example in batch:
        batch_sen1.append(example["sentence1"])
        batch_sen2.append(example["sentence2"])
        batch_labels.append(int(example["label"]))#从字符串转换为整数
    tokenized_batch = tokenizer(        
        batch_sen1,
        batch_sen2,
        padding=True,
        truncation=True,
        return_tensors="pt",
    )
    tokenized_batch["labels"] = torch.tensor(batch_labels) #将列表转换为张量
    return tokenized_batch
#创建train_dataloader传入traindata和collate_fn

    


In [6]:
train_dataloader = DataLoader(
        traindata,
        batch_size=4,
        shuffle=True,
        collate_fn=collate_fn,
    )
valid_dataloader= DataLoader(valdata, batch_size=4, shuffle=False, collate_fn=collate_fn)

In [19]:
#查看train_dataloader的第一个batch
next(iter(train_dataloader))

{'input_ids': tensor([[ 101, 5709, 1446, 7583, 2428,  802,  749, 6820, 1377,  809, 4500, 1408,
          102, 1963, 3362, 2769, 6206, 3118,  802, 4638, 7032, 7583, 6631, 6814,
         5709, 1446, 7583, 2428, 6820, 1377,  809, 4500, 5709, 1446,  720,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 2990, 4850, 2644, 4638, 6572, 2787, 3257, 3198,  679, 5016, 1394,
         2458, 6858, 3118,  802, 2140, 5709, 1446,  928, 4500, 1305, 3119, 7178,
          102, 2458, 6858, 5709, 1446,  928, 4500, 1305, 8024, 3119, 3621, 2990,
         4850, 6572, 2787, 7444, 6206,  784,  720, 3340,  816,  102],
        [ 101, 2769, 4638, 5709, 1446, 4500, 1071,  800, 3175, 2466, 2582,  720,
         6820, 3621,  102, 2582,  720, 6820, 1762, 5709, 1446, 2521, 6820, 3621,
         7027,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 2769, 5709, 1446, 6874, 3309,  7

In [7]:
#打印各个特征的shape
for batch in train_dataloader:
    print(batch["input_ids"].shape)
    print(batch["attention_mask"].shape)
    print(batch["token_type_ids"].shape)
    print(batch["labels"].shape)
    break

torch.Size([4, 42])
torch.Size([4, 42])
torch.Size([4, 42])
torch.Size([4])


如上，按照4个样本一个批次进行编码,每次的长度都不同，因为自动对每个 batch 中的样本进行补全和截断，**这种只在一个 batch 内进行补全的操作被称为动态补全 (Dynamic padding)**

# 3.构建模型并进行训练
- 先前使用的是AutoModelForSequenceClassification 类来完成。
- 在这里我们自己利用transformer库或BertForSequenceClassification类继承进行编写
    以Bert双向编码器为底座，任务类型是成对处理，只用 BERT 的“[CLS]”class token 向量来做下游决策，取最后一层 [CLS] 对应的 768 维（或 1024 维）向量，就能当整段文本的“句向量”用，再接个全连接层就能做分类或打分。
- 所以取最后一层 [CLS] 对应的向量，作为整段文本的“句向量”，再接个全连接层，就能做分类或打分。


### 1)继承Module类的写法，本质上在结构上增加一个model head


In [8]:
from torch import nn
from transformers import AutoModel

device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

class BertForPairwiseCLS(nn.Module):
    def __init__(self,dropout_prob=0.1):
        super(BertForPairwiseCLS,self).__init__()
        # 加载预训练的BERT模型
        self.bert_encoder=AutoModel.from_pretrained(checkpoint)
        #手动添加dropout层和分类层
        self.dropout=nn.Dropout(dropout_prob)
        self.classifier=nn.Linear(768,2)
    def forward(self,x):
        bert_output=self.bert_encoder(**x) #传入的x是一个字典，包含了输入的文本和注意力掩码
        cls_output=bert_output.last_hidden_state[:,0,:] #取最后一层隐藏层的第一个token的输出，即[CLS]对应的向量
        cls_output=self.dropout(cls_output)
        logits=self.classifier(cls_output)
        return logits

model=BertForPairwiseCLS().to(device)

model


cuda


BertForPairwiseCLS(
  (bert_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

来描述下模型中数据的变化维度：
- 初始输入
    text_a = "怎么查话费"  
    text_b = "如何查询手机余额"  
- 分词后：inputs = tokenizer(text_a, text_b, return_tensors='pt') 后
    维度是(1, 128)，其中128是最大序列长度，padding后的序列长度
    按照batch_size=4加载，则维度是(4, 128)
- embedding后：经过嵌入层后，维度是(4, 128, 768)，其中4是batch_size，12 8是最大序列长度，768是BERT的隐藏层维度
- 经过模型计算：
    经过注意力计算后对最后一层隐藏层的输出，维度是(4, 128, 768)
- 线性变换得到logits，维度是(4, 2)，其中2是分类数
- 对logits进行softmax，维度不变，得到每个类别的概率分布




## 2)继承BERT 模型（BertPreTrainedModel 类）的写法
- 所有参数都被继承了，包括模型的结构、参数、方法等
- 并可在config中自定义模型的结构

In [9]:
from torch import nn
from transformers import BertPreTrainedModel,BertModel
from transformers import AutoConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

class BertForPairwiseCLS2(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config,add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(768, 2)
        self.post_init()
    def forward(self,x):
        bert_outputs=self.bert(**x)
        cls_output=bert_outputs.last_hidden_state[:,0,:]
        cls_output=self.dropout(cls_output)
        logits=self.classifier(cls_output)
        return logits


config=AutoConfig.from_pretrained(checkpoint)

model=BertForPairwiseCLS2(config)
model.to(device)
model


cuda


BertForPairwiseCLS2(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [30]:
#将一个 Batch 的数据送入模型
 #先去掉labels
batch.pop('labels', None)   # 或者 del batch['labels']
y=model(batch)
print(y.size())

torch.Size([4, 2])


## 3）训练模型（微调参数）
- 学习率需要动态性：大学习率负责“快速找到大致正确的区域”，小学习率负责“在好区域里精细收敛”。
- 如果全程都用一个固定学习率：
    太大 → 来回震荡，收不住
    太小 → 走得太慢，甚至走不出局部差解

In [10]:
for step,batch in enumerate(train_dataloader,start=1):
        batch=batch.to(device)
        y=batch.pop('labels', None) 
        print(y)
        break

tensor([1, 0, 0, 1], device='cuda:0')


In [11]:
from tqdm.auto import tqdm # 用于显示训练进度条


In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,scheduler,epoch,total_loss):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_step_num = (epoch-1)*len(dataloader)

    model.train()
    for step,batch in enumerate(dataloader,start=1):
        batch=batch.to(device)
        y=batch.pop('labels', None) 
        pred=model(batch)
        loss = loss_fn(pred, y)

        optimizer.zero_grad() # 清空梯度
        loss.backward() # 计算梯度
        optimizer.step() # 更新模型参数
        scheduler.step() # 更新学习率,动态调整学习率

        total_loss += loss.item() # 累加损失
        progress_bar.set_description(f'loss: {total_loss/(finish_step_num + step):>7f}')
        progress_bar.update(1)
    return total_loss

def test_loop(dataloader, model, mode='Valid'):
    assert mode in ['Test', 'Valid'], "mode must be 'Test' or 'Valid'"
    size = len(dataloader.dataset)
    correct = 0

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            batch=batch.to(device)
            y=batch.pop('labels', None) 
            pred=model(batch)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    correct /= size
    print(f"{mode} Accuracy: {(100*correct):>0.1f}%")
    return correct
    


In [13]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

默认情况下，优化器会线性衰减学习率，对于上面的例子，学习率会线性地从 5e-5
 降到 0。
- 为了正确地定义学习率调度器，我们需要知道总的训练步数 (step)，它等于训练轮数 (Epoch number) 乘以每一轮中的步数（也就是训练 dataloader 的大小）：
因为每一步都会调用优化器，所以整个训练过程中，优化器会被调用多少次 optimizer.step()，也就是学习率会被更新多少次

In [14]:
# 计算这个训练过程需要的总步数
#len(train_dataloader) 已经“除过 batch_size 了
# ”DataLoader 的本质是：
# 把样本按 batch_size 切分后，形成一个 batch 的序列

print(f"traindata 有 {len(traindata)} 个样本")
epochs = 3
print(f"train_dataloader 有 {len(train_dataloader)} 个 batch")
num_training_steps = epochs * len(train_dataloader)

print(f"更新学习率需要 {num_training_steps} 步")


traindata 有 8000 个样本
train_dataloader 有 2000 个 batch
更新学习率需要 6000 步


In [15]:
#在这么多step，可以使得学习率线性地从初始 lr 下降到 0
from transformers import get_scheduler

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


将”训练循环”和”验证/测试循环”组合成 Epoch，就可以进行模型的训练和验证了
- 每个 Epoch 包含一个训练循环和一个验证/测试循环
- 训练循环用于更新模型参数，验证/测试循环用于评估模型性能
- 每个 Epoch 结束后，根据验证/测试准确率选择最佳模型

In [17]:
from torch.optim import AdamW

learning_rate = 1e-5
epoch_num = 3

loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=learning_rate)

total_loss = 0.
best_acc = 0.

for ep in range(epoch_num):
    print(f"Epoch {ep+1}/{epoch_num}\n-------------------------------")
    total_loss = train_loop(train_dataloader, model, loss_fn, optimizer, lr_scheduler, ep+1, total_loss)
    test_acc = test_loop(valid_dataloader, model, mode='Valid')
    
    if best_acc < test_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), f'epoch_{ep+1}_valid_acc_{(100*test_acc):0.1f}_model_weights.bin')
print("Done!")

Epoch 1/3
-------------------------------


loss: 0.669757:   9%|▉         | 182/2000 [00:11<01:52, 16.22it/s]

KeyboardInterrupt: 

loss: 0.669757:   9%|▉         | 183/2000 [00:26<01:52, 16.22it/s]

最终准确率在67.8%

# 4.加载模型并报告最终准确率

In [ ]:
model.load_state_dict(torch.load('epoch_3_valid_acc_74.1_model_weights.bin'))
test_loop(valid_dataloader, model, mode='Test')